In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [12]:
def lam(x, high, nominal,total_time):
    part = total_time/5
    if x < part:
        return np.random.normal(nominal, nominal/10)
    if x < 2*part:
        aux = (nominal-high)/(-part)*x + nominal + (nominal-high)
        return np.random.normal(aux, aux/10)
    if x < 3*part: 
        return high
    if x < 4*part:
        aux = -(high-nominal)/(part)*x + high+(high-nominal)*3
        return np.random.normal(aux, aux/10)
    else:
        return np.random.normal(nominal, nominal/10)

In [13]:
def generate_routefile(parent_dir):
    """Returns XML file specifying network layout for sumo simulation"""

    N = 7000  # number of time steps

    nominal = 1
    high =2 # At rush hour two times more cars

    # demand per second from different directions

    pEW = 1 / 20
    pNS = 1 / 80
    pWE = 1 / 20
    pSN = 1 / 80

    part = N/5


    if parent_dir == None:
        route_file_dir = "./network/cross.rou.xml"
    else:
        route_file_dir = os.path.join(parent_dir,"cross.rou.xml")

    with open(route_file_dir, "w") as routes:
        print("""<routes>
        <vType id="car" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2.5" maxSpeed="16.67" guiShape="passenger"/>
        <route id="right" edges="51o 1i 2o 52i" />
        <route id="left" edges="52o 2i 1o 51i" />
        <route id="down" edges="54o 4i 3o 53i" />
        <route id="up" edges="53o 3i 4o 54i" />""", file=routes)
        vehNr = 0
        
        for i in range(N):
            if random.uniform(0, 1) < pWE*lam(i,high,nominal,N):
                print('    <vehicle id="right_%i" type="car" route="right" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pEW*lam(i,high,nominal,N):
                print('    <vehicle id="left_%i" type="car" route="left" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pNS*lam(i,high,nominal,N):
                print('    <vehicle id="down_%i" type="car" route="up" depart="%i" color="1,0,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pSN*lam(i,high,nominal,N):
                print('    <vehicle id="UP_%i" type="car" route="down" depart="%i" color="1,0,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
        print("</routes>", file=routes)

In [14]:
generate_routefile("./")